In [1]:
import cv2
import os

def extract_frames(video_path, output_folder="frames", interval=10):
    os.makedirs(output_folder, exist_ok=True)

    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    saved_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Save every Nth frame
        if frame_count % interval == 0:
            filename = os.path.join(output_folder, f"frame_{saved_count:03}.jpg")
            cv2.imwrite(filename, frame)
            saved_count += 1

        frame_count += 1

    cap.release()
    print(f"✅ Done! Extracted {saved_count} frames to: {output_folder}")




In [2]:
extract_frames("gameplay.mp4", output_folder="frames", interval=10)

✅ Done! Extracted 581 frames to: frames


In [4]:
import torch

In [15]:
import torch.nn as nn
import torch.nn.functional as F

class DiseaseCNN(nn.Module):
    def __init__(self):
        super(DiseaseCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(32 * 56 * 56, 128)
        self.fc2 = nn.Linear(128, 2)  # Output logits for 2 classes

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # [B, 16, 112, 112]
        x = self.pool(F.relu(self.conv2(x)))  # [B, 32, 56, 56]
        x = x.view(x.size(0), -1)             # Flatten
        x = F.relu(self.fc1(x))
        return self.fc2(x)


In [16]:
disease_model = torch.load("/content/my_disease_classifier.pth", map_location="cpu", weights_only=False)

torch.save(disease_model.state_dict(), "/content/disease_weights.pth")
print("✅ Model weights saved successfully.")

✅ Model weights saved successfully.


In [25]:
# Rename it, since it's already a state_dict
import shutil
shutil.copy("/content/best_model.pth", "/content/severity_weights.pth")
print("✅ Already state_dict — copied as severity_weights.pth")

✅ Already state_dict — copied as severity_weights.pth


In [26]:
class FastViTMultiTaskModel(nn.Module):
    def __init__(self, region_class_names, muscle_class_names, severity_class_names, backbone_name="fastvit_sa12"):
        super(FastViTMultiTaskModel, self).__init__()

        # Store class name lists
        self.region_class_names = region_class_names
        self.muscle_class_names = muscle_class_names
        self.severity_class_names = severity_class_names

        # Number of classes
        region_classes = len(region_class_names)
        muscle_classes = len(muscle_class_names)
        severity_classes = len(severity_class_names)

        # ✅ Safety assertions
        assert region_classes > 1, "Region class list must have at least 2 classes"
        assert muscle_classes > 1, "Muscle class list must have at least 2 classes"
        assert severity_classes > 1, "Severity class list must have at least 2 classes"

        # 🔥 Load FastViT backbone from timm (no classification head)
        self.backbone = timm.create_model(backbone_name, pretrained=True, num_classes=0, global_pool='avg')

        # 📐 Determine feature dimension based on model variant
        if "t8" in backbone_name:
            feature_dim = 512
        elif "s12" in backbone_name:
            feature_dim = 768
        else:
            feature_dim = self.backbone.num_features  # Fallback

        # 🧠 Output heads
        self.region_head = nn.Linear(feature_dim, region_classes)
        self.muscle_head = nn.Linear(feature_dim, muscle_classes)
        self.severity_head = nn.Linear(feature_dim, severity_classes)

    def forward(self, x):
        features = self.backbone(x)  # Output shape: [B, feature_dim]

        region_out = self.region_head(features)      # [B, num_region_classes]
        muscle_out = self.muscle_head(features)      # [B, num_muscle_classes]
        severity_out = self.severity_head(features)  # [B, num_severity_classes]

        return region_out, muscle_out, severity_out

    # ✅ Add helper to get class name from prediction safely
    def get_class_names(self, region_pred, muscle_pred, severity_pred):
        def safe_get(lst, index):
            return lst[index] if index < len(lst) else f"[Invalid index {index}]"

        return (
            safe_get(self.region_class_names, region_pred),
            safe_get(self.muscle_class_names, muscle_pred),
            safe_get(self.severity_class_names, severity_pred)
        )


In [19]:
import torch.nn as nn
import torch.nn.functional as F

class DiseaseCNN(nn.Module):
    def __init__(self):
        super(DiseaseCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(32 * 56 * 56, 128)
        self.fc2 = nn.Linear(128, 2)  # Output logits for 2 classes

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # [B, 16, 112, 112]
        x = self.pool(F.relu(self.conv2(x)))  # [B, 32, 56, 56]
        x = x.view(x.size(0), -1)             # Flatten
        x = F.relu(self.fc1(x))
        return self.fc2(x)


In [33]:
import os, cv2
import timm
import torch
from collections import Counter
from PIL import Image
from torchvision import transforms
import torch.nn.functional as F
from torch.serialization import add_safe_globals
import pandas as pd
from sentence_transformers import SentenceTransformer, util

# ─────────────────────────────── SETUP ───────────────────────────────
video_path = "gameplay.mp4"
frame_dir = "/content/frames"
os.makedirs(frame_dir, exist_ok=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ─────────────────────────────── Load Disease Model ───────────────────────────────

disease_model.eval()
print("✅ Disease model loaded")

# ─────────────────────────────── Load FastViT Model ───────────────────────────────
add_safe_globals([FastViTMultiTaskModel])
df = pd.read_csv("/content/labels (3).csv")

region_classes = sorted(df['region'].unique().tolist())
muscle_classes = sorted(df['muscle'].unique().tolist())
severity_classes = ['mild', 'moderate', 'severe']

model = FastViTMultiTaskModel(
    region_class_names=region_classes,
    muscle_class_names=muscle_classes,
    severity_class_names=severity_classes,
    backbone_name="fastvit_sa12"
).to(device)

severity_weights = torch.load("/content/severity_weights.pth", map_location=device)
model.load_state_dict(severity_weights)
model.eval()
print("✅ Severity model loaded")

class_names = {
    "region_classes": region_classes,
    "muscle_classes": muscle_classes,
    "severity_classes": severity_classes
}

# ─────────────────────────────── Game Knowledge Base ───────────────────────────────
game_data = {
    "stroke": [
        {"title": "Finger Tap Challenge", "severity": "mild", "link": "https://youtu.be/VB2SD0sLk-8"},
        {"title": "Ball Rolling", "severity": "mild", "link": "https://youtu.be/KhDEfjbRJlU"},
        {"title": "Wrist Rotation Game", "severity": "mild", "link": "https://youtu.be/UeyJOvUskgE"},
        {"title": "Grip Strength Builder", "severity": "moderate", "link": "https://youtu.be/WqkdypN1WkY"},
        {"title": "Stacking Cups", "severity": "moderate", "link": "https://youtu.be/3Q0uKxl6kYQ"},
        {"title": "Arm Lifts with Bands", "severity": "moderate", "link": "https://youtu.be/nOSer6srdSA"},
        {"title": "Mirror Therapy", "severity": "severe", "link": "https://youtu.be/smr0IPRt7Uc"},
        {"title": "Sponge Squeeze", "severity": "severe", "link": "https://youtu.be/yMDKHDVdI4E"},
        {"title": "Visual Tracking", "severity": "severe", "link": "https://youtu.be/FmEHTWYyKvE"},
    ],
    "cp": [
        {"title": "Color Match Tap", "severity": "mild", "link": "https://youtu.be/g1A1J2p06j4"},
        {"title": "Ball Balance", "severity": "mild", "link": "https://youtu.be/ykqJxwqVKVE"},
        {"title": "Jump the Colors", "severity": "mild", "link": "https://youtu.be/eoZ89UwRXHI"},
        {"title": "Block Stack Game", "severity": "moderate", "link": "https://youtu.be/v-FUX1A95dg"},
        {"title": "Puzzle Builder", "severity": "moderate", "link": "https://youtu.be/Fn-fMbOmtjs"},
        {"title": "Reach the Target", "severity": "moderate", "link": "https://youtu.be/_FJ6qs3a91c"},
        {"title": "Soft Touch Stimulator", "severity": "severe", "link": "https://youtu.be/8WgmKr8yErE"},
        {"title": "Finger Tracing", "severity": "severe", "link": "https://youtu.be/FuAaUumUmjw"},
        {"title": "Shoulder Circle Game", "severity": "severe", "link": "https://youtu.be/UQHPY43JuwY"},
    ]
}

# ─────────────────────────────── RAG Preprocessing ───────────────────────────────
sentence_model = SentenceTransformer('all-MiniLM-L6-v2')
documents = []
metas = []

for disease_key, games in game_data.items():
    for game in games:
        doc = f"{game['title']} - rehab game for {disease_key.upper()} with {game['severity']} severity."
        documents.append(doc)
        metas.append({
            "disease": disease_key,
            "severity": game["severity"],
            "title": game["title"],
            "link": game["link"]
        })

doc_embeddings = sentence_model.encode(documents, convert_to_tensor=True)

def search_games_rag(query, disease_filter, severity_filter, top_k=5):
    query_embedding = sentence_model.encode(query, convert_to_tensor=True)
    cosine_scores = util.pytorch_cos_sim(query_embedding, doc_embeddings)[0]

    filtered = [
        (score.item(), meta)
        for score, meta in zip(cosine_scores, metas)
        if meta["disease"] == disease_filter and meta["severity"] == severity_filter
    ]
    top_matches = sorted(filtered, key=lambda x: x[0], reverse=True)[:top_k]
    return [
        f"{meta['title']} - Link: {meta['link']}"
        for _, meta in top_matches
    ]

# ─────────────────────────────── Inference Functions ───────────────────────────────
def predict_disease(img_path, model, device):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])
    image = Image.open(img_path).convert('RGB')
    img_tensor = transform(image).unsqueeze(0).to(device)
    with torch.no_grad():
        out = model(img_tensor)
        probs = F.softmax(out, dim=1)
        pred = torch.argmax(probs, dim=1).item()
        conf = probs[0][pred].item()
    return ["cp", "stroke"][pred], conf

def predict_severity(img_path, model, device, class_names):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])
    image = Image.open(img_path).convert('RGB')
    img_tensor = transform(image).unsqueeze(0).to(device)
    with torch.no_grad():
        region_out, muscle_out, severity_out = model(img_tensor)
        region = torch.argmax(region_out, 1).item()
        muscle = torch.argmax(muscle_out, 1).item()
        severity = torch.argmax(severity_out, 1).item()
        sev_conf = F.softmax(severity_out, 1)[0][severity].item()

    return {
        "region": class_names["region_classes"][region],
        "muscle": class_names["muscle_classes"][muscle],
        "severity": class_names["severity_classes"][severity],
        "severity_conf": sev_conf
    }

# ─────────────────────────────── MAIN PIPELINE ───────────────────────────────
frame_files = sorted(os.listdir(frame_dir), key=lambda x: int(''.join(filter(str.isdigit, x))))
print(f"✅ Found {len(frame_files)} frames in {frame_dir}")

all_severities = []

for fname in frame_files:
    fpath = os.path.join(frame_dir, fname)

    disease, d_conf = predict_disease(fpath, disease_model, device)
    result = predict_severity(fpath, model, device, class_names)

    print(f"\n🖼️ Frame: {fname}")
    print(f"📌 Disease : {disease} ({d_conf*100:.1f}%)")
    print(f"📍 Region  : {result['region']}")
    print(f"💪 Muscle  : {result['muscle']}")
    print(f"📊 Severity: {result['severity']} ({result['severity_conf']*100:.1f}%)")

    all_severities.append(result['severity'])



# ─────────────────────────────── FINAL RECOMMENDATION ───────────────────────────────
final_severity = Counter(all_severities).most_common(1)[0][0]
print(f"\n🔎 Most frequent severity: {final_severity.upper()}")

query = f"Suggest games for {disease.upper()} with {final_severity} severity"
recommended = search_games_rag(query, disease_filter=disease, severity_filter=final_severity, top_k=5)

print(f"\n🎮 Recommended VR Games for {disease.upper()} with {final_severity.upper()} severity:")
for game in recommended:
    print("✔️", game)


✅ Disease model loaded
✅ Severity model loaded
✅ Found 581 frames in /content/frames

🖼️ Frame: frame_000.jpg
📌 Disease : cp (51.2%)
📍 Region  : elbow
💪 Muscle  : triceps_brachii
📊 Severity: moderate (88.6%)

🖼️ Frame: frame_001.jpg
📌 Disease : cp (51.3%)
📍 Region  : forearm
💪 Muscle  : pronator_quadratus
📊 Severity: moderate (92.1%)

🖼️ Frame: frame_002.jpg
📌 Disease : cp (51.1%)
📍 Region  : forearm
💪 Muscle  : pronator_quadratus
📊 Severity: moderate (85.6%)

🖼️ Frame: frame_003.jpg
📌 Disease : cp (51.2%)
📍 Region  : forearm
💪 Muscle  : pronator_quadratus
📊 Severity: moderate (90.1%)

🖼️ Frame: frame_004.jpg
📌 Disease : cp (51.3%)
📍 Region  : forearm
💪 Muscle  : triceps_brachii
📊 Severity: moderate (90.5%)

🖼️ Frame: frame_005.jpg
📌 Disease : cp (51.3%)
📍 Region  : forearm
💪 Muscle  : pronator_quadratus
📊 Severity: moderate (86.1%)

🖼️ Frame: frame_006.jpg
📌 Disease : cp (51.4%)
📍 Region  : forearm
💪 Muscle  : pronator_quadratus
📊 Severity: moderate (85.0%)

🖼️ Frame: frame_007.jpg
📌 